
## Monitor HyperSolve Convergence

First, set up the HyperSolve solver in the correct working directory.  Set "case_directory" below.


In [ ]:
import json
import os

import infinity_plugins as infinity

infinity.initializeMPI()

case_directory = ...
os.chdir(case_directory)

with open("hypersolve.json") as f:
    config = json.load(f)

comm = infinity.getCommunicator()
mesh = infinity.import_mesh(config["preprocessor"], config["mesh filename"], comm)

hs_config = config["HyperSolve"]

hs_config["nonlinear solver settings"]["max time steps"] = 1
solver = infinity.PyFluidSolver("HyperSolve", json.dumps(hs_config), mesh, comm)




Now define a function to compute the residual norm of each equation


In [2]:

def calcEquationResidualNorms(hypersolve_solver):
    from math import sqrt
    equation_names = ["mass", "x-momentum", "y-momentum", "z-momentum", "total-energy"]
    eqn_l2_norms = []
    for name in equation_names:
        residual = hypersolve_solver.field("%s-residual" % name)
        l2 = 0
        for node_id in range(residual.size()):
            l2 += residual.at(node_id)[0]**2
        eqn_l2_norms.append(sqrt(l2))
    return eqn_l2_norms



Now run the main solver iteration loop, and use the above function to monitor the residual


In [3]:
from matplotlib import pyplot as plt
from ipywidgets import interactive
from IPython.display import display,clear_output


%matplotlib inline

max_steps = 100
iterations = []
eqn_l2_norms = []

for iteration in range(max_steps):
        solver.solve()
        plt.xlabel("iteration")
        plt.ylabel("Residual L2 Norm")
        plt.yscale("log")
        iterations.append(iteration)
        eqn_l2_norms.append(calcEquationResidualNorms(solver))
        plt.plot(iterations, eqn_l2_norms)
        plt.show()
        plt.clf()
        plt.cla()
        plt.close()
        clear_output(wait=True)


KeyboardInterrupt: 


Now shut down MPI


In [4]:
infinity.finalizeMPI()

